<a href="https://colab.research.google.com/github/yuzhentsai/Avalible_Mask/blob/main/available_mask.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [44]:
import requests
import json
import sqlite3
import datetime

利用 requests 對 API 來源發送一個請求

In [45]:
url = 'https://raw.githubusercontent.com/kiang/pharmacies/master/json/points.json'
response = requests.get(url)

將請求回應的內容存成一個字串格式

In [46]:
data = response.text

將長得像 json 格式的字串解析成字典或列表

In [47]:
raw_data = json.loads(data)
#print(raw_data)
Mask_db = sqlite3.connect("Mask_ava.db")
M = Mask_db.cursor()

In [48]:
# 新增且清空資料表 藥局
M.execute('''CREATE TABLE IF NOT EXISTS pharmacies
             (city text, counts text, createdAt datetime)''')
M.execute('''DELETE FROM pharmacies''')
Mask_db.commit()

In [49]:
med_amount = {}

計算各地區的藥局數量

In [50]:
# 填入欄位名稱
for data in raw_data["features"]:
    county = data["properties"]["county"]
    if not county :
      continue
    elif county not in med_amount:
         med_amount[county] = 0
    else:
      med_amount[county]+=1
#print(med_amount)

In [51]:
for key, value in med_amount.items():
    t = datetime.datetime.now()
    #print(f"INSERT INTO pharmacies VALUES ('{key}', '{value}', '{t}')")
    M.execute(f"INSERT INTO pharmacies VALUES ('{key}', '{value}', '{t}')")
    Mask_db.commit()

print('Number of pharmacies in each county:')
#print('\n'.join([f'{key}: {value}' for key, value in med_amount.items()]))
M.execute('SELECT * FROM pharmacies')
print('\n'.join([str(row) for row in M.fetchall()]))


Number of pharmacies in each county:
('臺北市', '338', '2024-08-21 22:33:28.958093')
('高雄市', '421', '2024-08-21 22:33:28.968392')
('臺中市', '419', '2024-08-21 22:33:28.977421')
('臺南市', '270', '2024-08-21 22:33:28.985797')
('基隆市', '56', '2024-08-21 22:33:28.994406')
('新竹市', '38', '2024-08-21 22:33:29.002922')
('嘉義市', '66', '2024-08-21 22:33:29.011919')
('新北市', '500', '2024-08-21 22:33:29.021754')
('桃園市', '263', '2024-08-21 22:33:29.030843')
('新竹縣', '45', '2024-08-21 22:33:29.040445')
('宜蘭縣', '75', '2024-08-21 22:33:29.049125')
('苗栗縣', '55', '2024-08-21 22:33:29.058147')
('彰化縣', '178', '2024-08-21 22:33:29.066757')
('南投縣', '66', '2024-08-21 22:33:29.075778')
('雲林縣', '128', '2024-08-21 22:33:29.085422')
('嘉義縣', '83', '2024-08-21 22:33:29.094517')
('屏東縣', '139', '2024-08-21 22:33:29.103238')
('澎湖縣', '10', '2024-08-21 22:33:29.112487')
('花蓮縣', '45', '2024-08-21 22:33:29.121943')
('臺東縣', '22', '2024-08-21 22:33:29.131158')
('金門縣', '5', '2024-08-21 22:33:29.141492')
('連江縣', '0', '2024-08-21 22:33:

In [52]:
#新增且清空資料表 口罩
M.execute('''CREATE TABLE IF NOT EXISTS masks
        (city text, counts text, createdAt datetime)''')
M.execute('DELETE FROM masks')
Mask_db.commit()
mask_amount = {}

In [53]:
# 填入欄位名稱
for data in raw_data["features"]:
    county = data["properties"]["county"]
    mask_adult = data["properties"]["mask_adult"]
    if county == "":
      continue
    elif county not in mask_amount:
         mask_amount[county] = 0
    else:
      mask_amount[county]+=mask_adult

# 將結果從大到小排列
mask_amount = dict(sorted(mask_amount.items(), key=lambda item: item[1],reverse=True))

#print(mask_amount)

In [54]:
for key, value in mask_amount.items():
    t = datetime.datetime.now()
    M.execute(f'INSERT INTO masks VALUES (?, ?, ?)', (key, value, t))
    Mask_db.commit()

print('\nNumber of adult masks (descending order) in each county:')
#print('\n'.join([f'{key}: {value}' for key, value in mask_amount.items()]))
M.execute('SELECT * FROM masks')
print('\n'.join([str(row) for row in M.fetchall()]))

Mask_db.commit()
Mask_db.close()


Number of adult masks (descending order) in each county:
('新北市', '670150', '2024-08-21 22:33:30.444466')
('臺中市', '535430', '2024-08-21 22:33:30.455157')
('高雄市', '424170', '2024-08-21 22:33:30.464594')
('臺南市', '392430', '2024-08-21 22:33:30.474535')
('桃園市', '330010', '2024-08-21 22:33:30.484453')
('臺北市', '322490', '2024-08-21 22:33:30.493603')
('彰化縣', '218090', '2024-08-21 22:33:30.503491')
('雲林縣', '195540', '2024-08-21 22:33:30.512790')
('屏東縣', '181480', '2024-08-21 22:33:30.521658')
('苗栗縣', '119500', '2024-08-21 22:33:30.531145')
('嘉義縣', '112190', '2024-08-21 22:33:30.542677')
('嘉義市', '105870', '2024-08-21 22:33:30.554315')
('宜蘭縣', '105830', '2024-08-21 22:33:30.563650')
('南投縣', '81680', '2024-08-21 22:33:30.574352')
('基隆市', '79840', '2024-08-21 22:33:30.585548')
('花蓮縣', '56570', '2024-08-21 22:33:30.595956')
('新竹縣', '51280', '2024-08-21 22:33:30.606602')
('新竹市', '45630', '2024-08-21 22:33:30.615178')
('臺東縣', '27720', '2024-08-21 22:33:30.624481')
('金門縣', '15510', '2024-08-21 22:33:3